In [1]:
import os
os.chdir('..')

In [10]:
import numpy as np
import pandas as pd

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler

from scripts.transformers_JuanPablo import CheckColumnNames,UnknownToZero,FixRanges

In [ ]:
data = pd.read_csv("data/datos_grasas_Tec.csv", encoding="latin1")
data_pdfs = pd.read_csv("data/datos_grasas_pdfs.csv", encoding="latin1")

,idDatosGrasas,codigoGrasa,Aceite Base,Espesante,Grado NLGI Consistencia,Viscosidad del Aceite Base a 40°C. cSt,"Penetración de Cono a 25°C, 0.1mm","Punto de Gota, °C","Estabilidad Mecánica, %","Punto de Soldadura Cuatro Bolas, kgf",...,Factor de Velocidad,"Temperatura de Servicio °C, min","Temperatura de Servicio °C, max",categoria,subtitulo,descripcion,beneficios,aplicaciones,color,textura
0,1,Grasa_1,Semi-Sintetico,Complejo Sulfonato de Calcio,2.0,680.0,279,304,1.07,500.0,...,200000.0,5,220,Grasa,Grasa de servicio pesado para alta resistencia...,El producto es grasa lubricante de servicio pe...,Excelentes caracterÃ­sticas de resistencia al ...,Debido a su composiciÃ³n Ã³ptima de aceites de...,CafÃ©,Suave


In [ ]:
process = Pipeline(steps=[
    ("To have columns names needed", CheckColumnNames()),
    ("To change unkown data to zeros", UnknownToZero("Grado NLGI Consistencia")),
    ("To fix ranges and single values", FixRanges("Penetración de Cono a 25°C, 0.1mm"))
])

In [ ]:
X=process.fit_transform(data)
X.head(1)

,idDatosGrasas,codigoGrasa,Aceite Base,Espesante,Grado NLGI Consistencia,Viscosidad del Aceite Base a 40°C. cSt,"Punto de Gota, °C","Estabilidad Mecánica, %","Punto de Soldadura Cuatro Bolas, kgf","Desgaste Cuatro Bolas, mm",...,"Temperatura de Servicio °C, max",categoria,subtitulo,descripcion,beneficios,aplicaciones,color,textura,"Penetración de Cono a 25°C, 0.1mm min","Penetración de Cono a 25°C, 0.1mm max"
0,1,Grasa_1,Semi-Sintetico,Complejo Sulfonato de Calcio,2.0,680.0,304,1.07,500.0,NaN,...,220,Grasa,Grasa de servicio pesado para alta resistencia...,El producto es grasa lubricante de servicio pe...,Excelentes caracterÃ­sticas de resistencia al ...,Debido a su composiciÃ³n Ã³ptima de aceites de...,CafÃ©,Suave,279,279


In [ ]:
Y=process.transform(data_pdfs)
Y.head(1)

In [ ]:
pipe = Pipeline(steps=[
    ('MinMax', ColumnTransformer(transformers=[('MinMax', MinMaxScaler(), slice(1,None))],remainder="passthrough")),
])